# ARC 2018 Model #

In [13]:
import openmc
import neutronics_material_maker as nmm
import math


## Materials ##

All material management is done using `neutronics_material_maker`. The name of each material must correspond to the name of the region of the DAGMC model to which it should be assigned.

In [12]:
nmm.AddMaterialFromFile("mat_lib.json")

# Material Parameters:
Li6_enrichment = 7.8
U235_enrichment = 0.007
percent_fertile = 5 / 100

plasma = nmm.Material.from_library("plasma")

flibe = nmm.Material.from_library("flibe", enrichment=Li6_enrichment)
uf4 = nmm.Material.from_library("uf4")

#Creating materials for all four FLiBe regions by mixing UF4 and FLiBe materials by volume
ib_tank = nmm.Material.from_mixture(name='inboard_tank', materials=[flibe, uf4], fracs=[1 - percent_fertile, percent_fertile], percent_type='vo')
ob_tank = nmm.Material.from_mixture(name='outboard_tank', materials=[flibe, uf4], fracs=[1 - percent_fertile, percent_fertile], percent_type='vo')

ib_cc = nmm.Material.from_mixture(name='inboard_cc', materials=[flibe, uf4], fracs=[1 - percent_fertile, percent_fertile], percent_type='vo')
ob_cc = nmm.Material.from_mixture(name='outboard_cc', materials=[flibe, uf4], fracs=[1 - percent_fertile, percent_fertile], percent_type='vo')

lead = nmm.Material.from_library("lead")

ib_mult = lead.openmc_material
ib_mult.name = 'inboard_multiplier'

ob_mult = lead.openmc_material
ob_mult.name = 'outboard_multiplier'
ob_mult.id = ib_mult.id + 1



Added materials to library from mat_lib.json
Added material ['plasma', 'flibe', 'uf4', 'tungsten', 'lead']


/home/jlball/.local/lib/python3.8/site-packages/openmc/mixin.py:67: IDWarning: Another Material instance already exists with id=99.
  warn(msg, IDWarning)


## Geometry ##

The geometry is built in a separate file which uses the Paramak code to build a DAGMC model. Here it is imported and then a bounding region is provided.

In [14]:
dag_univ = openmc.DAGMCUniverse("arc2018.h5m")

# creates an edge of universe boundary surface
vac_surf = openmc.Sphere(r=10000, surface_id=9999, boundary_type="vacuum")

# adds reflective surface for the sector model at 0 degrees
reflective_1 = openmc.Plane(
    a=math.sin(0),
    b=-math.cos(0),
    c=0.0,
    d=0.0,
    surface_id=9991,
    boundary_type="reflective",
)

# specifies the region as below the universe boundary and inside the reflective surfaces
region = -vac_surf & -reflective_1 

# creates a cell from the region and fills the cell with the dagmc geometry
containing_cell = openmc.Cell(cell_id=9999, region=region, fill=dag_univ)

geometry = openmc.Geometry(root=[containing_cell])

## Settings ##

A simple line source geometry is defined.